In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Ch04").getOrCreate()
spark.conf.set("spark.driver.memory", "6g")
sc = spark.sparkContext

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

In [3]:
colNames = ["Elevation", "Aspect", "Slope",
"Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"]
for i in range(4):
    colNames += ["Wilderness_Area_"+str(i),]
for i in range(40):
    colNames += ["Soil_Type_"+str(i),]
colNames += ["Cover_Type",]

In [4]:
schema = StructType()
for name in colNames:
    if name == "Cover_Type":
        schema.add(StructField(name, DoubleType(), True))
    else:
        schema.add(StructField(name, IntegerType(), True))

In [7]:
data = spark.read.csv("./data/covtype.data", header=False, schema=schema)
data = data.sample(0.7)

In [8]:
data.printSchema()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area_0: integer (nullable = true)
 |-- Wilderness_Area_1: integer (nullable = true)
 |-- Wilderness_Area_2: integer (nullable = true)
 |-- Wilderness_Area_3: integer (nullable = true)
 |-- Soil_Type_0: integer (nullable = true)
 |-- Soil_Type_1: integer (nullable = true)
 |-- Soil_Type_2: integer (nullable = true)
 |-- Soil_Type_3: integer (nullable = true)
 |-- Soil_Type_4: integer (nullable = true)
 |-- Soil_Type_5: integer (nullable = true)
 |-- Soil_Type

In [9]:
data.take(1)

[Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area_0=1, Wilderness_Area_1=0, Wilderness_Area_2=0, Wilderness_Area_3=0, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5.0)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
(trainData, testData) = data.randomSplit([0.9, 0.1])

In [12]:
inputCols = trainData.drop('Cover_Type').columns

In [13]:
assembler = VectorAssembler(
    inputCols=inputCols,
    outputCol="featureVector")
assembledTrainData = assembler.transform(trainData)
assembledTrainData.select('featureVector').show(truncate=False)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175.0,195.0,224.0,168.0,732.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1899.0,355.0,22.0,153.0,43.0,124.0,178.0,195.0,151.0,819.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1901.0,311.0,9.0,30.0,2.0,190.

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier
classifier = DecisionTreeClassifier(labelCol="Cover_Type", featuresCol="featureVector", predictionCol="prediction")
model = classifier.fit(assembledTrainData)

print(model.toDebugString)
print(model.featureImportances)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_45fd98cbd365066ceb77) of depth 5 with 63 nodes
  If (feature 0 <= 3036.5)
   If (feature 0 <= 2555.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2448.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2448.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 22 <= 0.5)
      If (feature 9 <= 4604.5)
       Predict: 2.0
      Else (feature 9 > 4604.5)
       Predict: 2.0
     Else (feature 22 > 0.5)
      If (feature 9 <= 1093.5)
       Predict: 2.0
      Else (feature 9 > 1093.5)
       Predict: 2.0
   Else (feature 0 > 2555.5)
    If (feature 15 <= 0.5)
     If (feature 0 <= 2939.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 0 > 2939.5)
      If (feature 3 <= 142.0)
       Predict:

In [15]:
predictions = model.transform(assembledTrainData)
predictions.select(["Cover_Type", "prediction", "probability"]).show(truncate=False)

+----------+----------+-------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                      |
+----------+----------+-------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.0326005308227753,0.6326305773949622,0.05082878461615504,0.0,0.28394010716610746,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.0326005308227753,0.6326305773949622,0.05082878461615504,0.0,0.28394010716610746,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.0326005308227753,0.6326305773949622,0.05082878461615504,0.0,0.28394010716610746,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.0326005308227753,0.6326305773949622,0.05082878461615504,0.0,0.28394010716610746,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.0326005308227753,0.6326305773949622,0.05082878461615504,0.0,0.28394010716610746,0.0]  |
|6.0       |3.0       |[

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction")

In [17]:
evaluator.setMetricName("accuracy").evaluate(predictions)

0.6964547887828391

In [18]:
evaluator.setMetricName("f1").evaluate(predictions)

0.6795884301953133

#### confusion matrix - not supported in PySpark ML library

In [19]:
from pyspark.ml import Pipeline

inputCols = trainData.columns[:-1]
assembler = VectorAssembler(inputCols=inputCols, outputCol="featureVector")
classifier = DecisionTreeClassifier(labelCol="Cover_Type", featuresCol="featureVector", predictionCol="prediction")
pipeline = Pipeline(stages=[assembler, classifier])

In [20]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder()\
    .addGrid(classifier.impurity, ["gini", "entropy"])\
    .addGrid(classifier.maxDepth, [10, 30])\
    .addGrid(classifier.maxBins, [40, 100])\
    .addGrid(classifier.minInfoGain, [0.0, 0.05])\
    .build()

In [21]:
multiclassEval = MulticlassClassificationEvaluator(
    labelCol="Cover_Type",
    predictionCol="prediction",
    metricName="accuracy")
multiclassEval.evaluate(predictions)

0.6964547887828391

In [22]:
from pyspark.ml.tuning import TrainValidationSplit

validator = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=multiclassEval,
    trainRatio=0.9)
validatorModel = validator.fit(trainData)

In [23]:
bestModel = validatorModel.bestModel

In [24]:
bestModel.stages[-1].extractParamMap()

{Param(parent='DecisionTreeClassifier_432b9017f8fe9ad11f9a', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='DecisionTreeClassifier_432b9017f8fe9ad11f9a', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='DecisionTreeClassifier_432b9017f8fe9ad11f9a', name='featuresCol', doc='features column name'): 'featureVector',
 Param(parent='DecisionTreeClassifier_432b9017f8fe9ad11f9a', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy',
 Param(parent='DecisionTreeClassifier_432b9017f8fe9ad11f9a',

In [25]:
paramsAndMetrics = validatorModel.validationMetrics
paramsAndMetrics

[0.775266360157476,
 0.6723012966990612,
 0.7809101671117474,
 0.6711450045425763,
 0.9215097871871817,
 0.6723012966990612,
 0.92654791729758,
 0.6711450045425763,
 0.7726234066569392,
 0.7143958373482366,
 0.7717424221567601,
 0.713900283566886,
 0.9262175481100129,
 0.7231781515843956,
 0.9294661784544228,
 0.727197643366462]

In [26]:
multiclassEval.evaluate(bestModel.transform(testData))

0.9303621169916435

#### undoing the one-hot encoding

In [27]:
wildernessCols = []
for i in range(4):
    wildernessCols += ["Wilderness_Area_"+str(i),]

In [28]:
wildernessAssembler = VectorAssembler(
    inputCols=wildernessCols,
    outputCol="wilderness")

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType, StructType

unhotudf = udf(lambda x: float(x.toArray().nonzero()[0]), DoubleType())

In [30]:
withWilderness = wildernessAssembler.transform(data)
withWilderness = withWilderness\
    .drop(*wildernessCols)\
    .withColumn("wilderness", unhotudf(withWilderness['wilderness']))
withWilderness.take(1)


[Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5.0, wilderness=0.0)]

In [31]:
soilCols = []
for i in range(40):
    soilCols += ["Soil_Type_"+str(i),]


In [32]:
soilAssembler = VectorAssembler(
        inputCols=soilCols,
        outputCol="soil")

withWilderness = soilAssembler.transform(withWilderness)
unencData = withWilderness\
    .drop(*soilCols)\
    .withColumn("soil", unhotudf(withWilderness['soil']))
unencData.take(1)

[Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Cover_Type=5.0, wilderness=0.0, soil=28.0)]

#### Decision Tree Classifier with unencoded data

In [51]:
(unencTrainData, unencTestData) = unencData.randomSplit([0.9, 0.1])

In [52]:
from pyspark.ml.feature import VectorIndexer

inputCols = unencTrainData.drop('Cover_Type').columns
assembler = VectorAssembler(
    inputCols=inputCols,
    outputCol="featureVector")
indexer = VectorIndexer(
    maxCategories=40,
    inputCol="featureVector",
    outputCol="indexedVector")
classifier = DecisionTreeClassifier(
    seed=42,
    labelCol="Cover_Type",
    featuresCol="indexedVector",
    predictionCol="prediction")
pipeline = Pipeline(stages=[assembler, indexer, classifier])

#### Random Forest Classifier

In [53]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(
    seed=42,
    maxBins=40,
    labelCol="Cover_Type",
    featuresCol="indexedVector",
    predictionCol="prediction")
pipeline = Pipeline(stages=[assembler, indexer, classifier])

In [70]:
paramGrid = ParamGridBuilder()\
    .addGrid(classifier.minInfoGain, [0.01, 0.1])\
    .addGrid(classifier.numTrees, [1, 20])\
    .addGrid(classifier.featureSubsetStrategy, [10,20])\
    .build()

In [63]:
multiclassEval = MulticlassClassificationEvaluator(
    labelCol="Cover_Type",
    predictionCol="prediction",
    metricName="accuracy")

In [64]:
validator = TrainValidationSplit(
    seed=42,
    estimator=pipeline,
    evaluator=multiclassEval,
    estimatorParamMaps=paramGrid,
    trainRatio=0.9)

In [71]:
%%time
validatorModel = validator.fit(unencTrainData)
bestModel = validatorModel.bestModel
forestModel = bestModel.stages[-1]
print(forestModel.extractParamMap())

{Param(parent='RandomForestClassifier_45e9a5ace69b97fb1ebc', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False, Param(parent='RandomForestClassifier_45e9a5ace69b97fb1ebc', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10, Param(parent='RandomForestClassifier_45e9a5ace69b97fb1ebc', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto', Param(parent='RandomForestClassifier_45e9a5ace69b97fb1ebc', name='featuresCol', doc='features column name'): 'indexedVector', Param(parent='Rando

In [66]:
forestModel.getNumTrees

1

In [67]:
sorted(list(zip(inputCols, forestModel.featureImportances)), key=lambda x: x[1], reverse=True)

[('Elevation', 0.7923040942711657),
 ('soil', 0.11768096858410305),
 ('wilderness', 0.03845661669381131),
 ('Horizontal_Distance_To_Roadways', 0.02843022997776665),
 ('Horizontal_Distance_To_Fire_Points', 0.012640611525555063),
 ('Horizontal_Distance_To_Hydrology', 0.005253740128991872),
 ('Vertical_Distance_To_Hydrology', 0.0033425724201158395),
 ('Aspect', 0.001891166398490582),
 ('Slope', 0.0),
 ('Hillshade_9am', 0.0),
 ('Hillshade_Noon', 0.0),
 ('Hillshade_3pm', 0.0)]

In [72]:
testAccuracy = multiclassEval.evaluate(bestModel.transform(unencTestData))
testAccuracy

0.697731196054254

In [43]:
bestModel.transform(unencTestData.drop("Cover_Type")).show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------+----+--------------------+--------------------+--------------------+--------------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|wilderness|soil|       featureVector|       indexedVector|       rawPrediction|         probability|prediction|
+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------+----+--------------------+--------------------+--------------------+--------------------+----------+
|     1896|   337|   12|                              30|             

### Logisitic Regression

In [45]:
(trainData, testData) = data.randomSplit([0.9, 0.1])

inputCols = trainData.drop('Cover_Type').columns

assembler = VectorAssembler(
    inputCols=inputCols,
    outputCol="featureVector")
assembledTrainData = assembler.transform(trainData)
assembledTrainData.select('featureVector').show(truncate=False)


from pyspark.ml.classification import LogisticRegression
classifier = LogisticRegression(labelCol="Cover_Type", featuresCol="featureVector", predictionCol="prediction")
model = classifier.fit(assembledTrainData)

#print(model.toDebugString)
#print(model.featureImportances)

predictions = model.transform(assembledTrainData)
predictions.select(["Cover_Type", "prediction", "probability"]).show(truncate=False)


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction")


+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175.0,195.0,224.0,168.0,732.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1899.0,355.0,22.0,153.0,43.0,124.0,178.0,195.0,151.0,819.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1901.0,311.0,9.0,30.0,2.0,190.

In [ ]:
evaluator.setMetricName("accuracy").evaluate(predictions)
evaluator.setMetricName("f1").evaluate(predictions)

### Logistic Regression - ParamGrid

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(classifier.regParam, [0.1, 0.2])\
    .addGrid(classifier.elasticNetParam, [0.001, 0.01])\
    .build()

In [ ]:
multiclassEval = MulticlassClassificationEvaluator(
    labelCol="Cover_Type",
    predictionCol="prediction",
    metricName="accuracy")

In [ ]:
validator = TrainValidationSplit(
    estimator=pipeline,
    evaluator=multiclassEval,
    estimatorParamMaps=paramGrid,
    trainRatio=0.9)

In [ ]:
%%time
validatorModel = validator.fit(unencTrainData)
bestModel = validatorModel.bestModel
forestModel = bestModel.stages[-1]
print(forestModel.extractParamMap())

In [ ]:
testAccuracy = multiclassEval.evaluate(bestModel.transform(unencTestData))
testAccuracy